In [ ]:
"""Calculates the least effective time covered and writes the maximum coverage to an output file"""
def max_coverage(intervals,temp_intervals,file_path):    
    import pandas as pd
    import numpy as np
    coverage =0
    for i in range(len(intervals)):
        p, n1, n2=0, 0, 0
    # Checking overlap with previous and next interval to calculate the alone time of a lifeguard in every interval.
        if i == 0:
            if pd.Interval(intervals[i][0],intervals[i][1]).overlaps(pd.Interval(intervals[i+1][0],intervals[i+1][1])):
                if(intervals[i][1]>intervals[i+1][1]):
                    intervals[i].extend([abs(intervals[i][1] - intervals[i][0]) - abs(intervals[i+1][1] - intervals[i+1][0])])
                else: 
                    intervals[i].extend([abs(intervals[i][1] - intervals[i][0]) - abs(intervals[i][1] - intervals[i+1][0])])
                i = i+1
            else:
                intervals[i].extend([abs(intervals[i][1] - intervals[i][0])])
                i = i+1        

        elif i == len(intervals) - 1:
            if intervals[i][1] < intervals[i-1][1]:
                intervals[i].extend([0])
                i = i+1
            elif pd.Interval(intervals[i-1][0],intervals[i-1][1]).overlaps(pd.Interval(intervals[i][0],intervals[i][1])):
                intervals[i].extend([abs(intervals[i][1] - intervals[i][0]) - abs(intervals[i-1][1] - intervals[i][0])])
                i = i+1
            else:
                intervals[i].extend([abs(intervals[i][1] - intervals[i][0])])
                i = i+1

        else:
            if intervals[i][1] < intervals[i-1][1]:
                intervals[i].extend([0])
                i = i+1
                continue
            if intervals[i][0] < intervals[i-1][1] and intervals[i][1] > intervals[i-1][1]:
                previous_overlap = intervals[i-1][1] - intervals[i][0]
                intervals[i].extend([abs(intervals[i][1] - intervals[i][0]) - previous_overlap])
                p = 1
            if intervals[i+1][0] < intervals[i][1] and intervals[i+1][1] > intervals[i][1]:
                next_overlap =  intervals[i][1] - intervals[i+1][0]
                if p == 1 and intervals[i+1][0]< intervals[i-1][1]:
                    intervals[i][2] = intervals[i][2] - (intervals[i][1] - intervals[i-1][1])
                elif p==1 and intervals[i+1][0]> intervals[i-1][1]:
                    intervals[i][2] = intervals[i][2] - (intervals[i][1] - intervals[i+1][0])
                else:
                    intervals[i].extend([abs(intervals[i][1] - intervals[i][0]) - next_overlap])
                n1 = 1
            if intervals[i+1][0] < intervals[i][1] and intervals[i+1][1] < intervals[i][1]:
                    if p==1 and intervals[i+1][1] < intervals[i-1][1]:
                        intervals[i][2] = intervals[i][2]
                    elif p == 1 and intervals[i+1][1] > intervals[i-1][1]:
                        next_overlap = intervals[i+1][1] - intervals[i-1][1]
                        intervals[i][2] = intervals[i][2] - next_overlap
                    else:
                        intervals[i].extend([intervals[i][1] - intervals[i+1][1] + intervals[i+1][0]-intervals[i][0]])
                    n2=1
            if p==1 or n1==1 or n2==1:
                i = i+1
            else:
                intervals[i].extend([abs(intervals[i][1] - intervals[i][0])])
                i = i+1

    # Interval with least effective time covered by a lifeguard
    least_effective_interval = min(intervals, key=lambda x: x[2])
    least_effective_coverage = least_effective_interval[2]
    
    # Maximum coverage of lifeguards
    ind = np.where(np.diff(np.array(temp_intervals).flatten()) <= 0)[0]
    merged_intervals = np.delete(temp_intervals, [ind, ind+1]).reshape(-1, 2)
    for i in range(len(merged_intervals)):
        coverage = coverage + merged_intervals[i][1] - merged_intervals[i][0]
        i = i+1
    output = coverage - least_effective_coverage
    file_name = os.path.basename(file_path).split('.')[0] + ".out"
    with open(file_name, "w") as external_file:
        print(output, file=external_file)
    external_file.close()

"""Read all files in a folder"""
import os
path ="D:\PYTHON_PRO_1\DataStructuresAssignment\data"
os.chdir(path)
for file in os.listdir():
    if file.endswith(".in"):
        file_path = f"{path}\{file}"
        intervals = []
        temp_intervals =[]
        with open(file_path) as file:
            next(file)
            for line in file:
                (s, e) = line.split()
                intervals.append([eval(s),eval(e)])
                temp_intervals.append((eval(s),eval(e)))
            intervals.sort()
            temp_intervals.sort()
            max_coverage(intervals,temp_intervals,file_path)